# Analysis of "Unfounded" Rape Reports

This notebook loads the data from the FBI's Uniform Crime Report. Then that data is used to calculate: the number of rapes declared "unfounded", the number of "actual" rapes, and the percentage of rapes declared "unfounded" at both the state and agency level. This is analysis is in support of the BuzzFeed News story ["When Detectives Dismiss Rape Reports Before Investigating Them"](https://www.buzzfeed.com/alexcampbell/unfounded) published on September 8, 2016.

The notebook is divided into sections. The first is dedicated to loading the UCR data, the next two deal with calculating the national "unfounded" rate (see those sections for the caveats that entails), and the remainder deals with the "unfounded" rate of individual police departments.

## Loading Data

All `ucr_files` used in this notebook from the FBI's Uniform Crime Report, via the National Archive of Criminal Justice Data (NACJD). The crosswalk data comes from the Department of Justice's Bureau of Justice Statistics, also via the NACJD. For more details, [see here](https://github.com/BuzzFeedNews/2016-08-ucr-analysis#data-sources).

In [1]:
import pandas as pd

In [2]:
agencies = pd.read_csv("../data/crosswalk/35158-0001-Data.tsv", sep="\t")\
    .set_index("ORI7")[[
          "NAME", "STATENAME", "COUNTYNAME", "LG_NAME", "ADDRESS_NAME", "ADDRESS_CITY", "ADDRESS_STATE"      
    ]]
agencies.head(2)

,NAME,STATENAME,COUNTYNAME,LG_NAME,ADDRESS_NAME,ADDRESS_CITY,ADDRESS_STATE
ORI7,,,,,,,
AL00402,AUTAUGAVILLE POLICE DEPARTMENT,ALABAMA,AUTAUGA,AUTAUGAVILLE TOWN,AUTAUGAVILLE POLICE DEPARTMENT,AUTAUGAVILLE,AL
AL00401,PRATTVILLE POLICE DEPARTMENT,ALABAMA,AUTAUGA,PRATTVILLE CITY,PRATTVILLE POLICE DEPARTMENT,PRATTVILLE,AL


In [3]:
ucr_files = [
    "30766-0001-Data.tsv", # 2009
    "33526-0001-Data.tsv", # 2010
    "34586-0001-Data.tsv", # 2011
    "35021-0001-Data.tsv", # 2012
    "36122-0001-Data.tsv", # 2013
    "36391-0001-Data.tsv", # 2014
]

In [4]:
# The `ucr` variable holds the "offenses known and clearances by arrest" data for 2009–2014
dtype_dict = {"V7": str, "V15": str, "V24": str, "V32": str}

ucr = pd.concat([ pd.read_csv("../data/ucr/" + f, sep="\t", dtype=dtype_dict)
    for f in ucr_files ]).reset_index(drop=True)

ucr.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V1439,V1440,V1441,V1442,V1443,V1444,V1445,V1446,V1447,V1448
0,1,1,ALAST00,8D,6,2009,70402,N,,6231986,...,0,0,0,0,0,0,0,0,0,0
1,1,1,ALDI003,7,6,2009,49797,N,,10142009,...,0,0,0,0,0,0,0,0,0,0


The function below takes advantage of the fact that each month in the UCR report is associated with exactly 118 variables. (For instance, column `V46` corresponds to "JAN: UNFOUNDED RAPE TOTL", and column `V164` corresponds to "FEB: UNFOUNDED RAPE TOTL".) The function uses this fact to find — given a variable's January column — the total sum of variable for the full year.

In [5]:
def get_totals(init_ix): 
    colnames = [ "V" + str(init_ix + 118*x) for x in range(12) ]
    return ucr[colnames].sum(axis=1)

In [6]:
# State code (`V2`) corresponds to the first two characters of the ORI code (`V3`)
assert((ucr.groupby(ucr["V3"].str.slice(0, 2))["V2"].nunique() != 1).sum() == 0)

In [7]:
totals = pd.DataFrame({
    "state_code": ucr["V2"],
    "ori_code": ucr["V3"],
    "year": ucr["V6"],
    "rape_declared_unfounded": get_totals(46), # "UNFOUNDED RAPE TOTL"
    "rape_declared_actual": get_totals(72) # "ACT NUM RAPE TOTL"
})
totals["rape_reported"] = totals["rape_declared_actual"] + totals["rape_declared_unfounded"]

In [8]:
totals.groupby([ totals["ori_code"].str.slice(0, 2) ])["state_code"].nunique().value_counts()

1    55
Name: state_code, dtype: int64

In [9]:
totals.head()

,ori_code,rape_declared_actual,rape_declared_unfounded,state_code,year,rape_reported
0,ALAST00,0,0,1,2009,0
1,ALDI003,2,0,1,2009,2
2,AL00100,46,1,1,2009,47
3,AL00101,24,0,1,2009,24
4,AL00102,198,0,1,2009,198


In [10]:
code_grps = totals.groupby(["ori_code", "year"])
_agency_totals_by_year = pd.DataFrame({
    "rape_declared_actual": code_grps["rape_declared_actual"].sum(),
    "rape_declared_unfounded": code_grps["rape_declared_unfounded"].sum(),
    "rape_reported": code_grps["rape_reported"].sum(),
})
_agency_totals_by_year["unfounded_pct"] = (100 * \
    _agency_totals_by_year["rape_declared_unfounded"] / _agency_totals_by_year["rape_reported"]).round(1)
agency_totals_by_year = _agency_totals_by_year.unstack()
agency_totals_by_year.head().T

ori_code                      AK00101  AK00102  AK00103  AK00104  AK00105
                        year                                             
rape_declared_actual    2009    282.0     59.0     25.0      8.0      2.0
                        2010    264.0     70.0     20.0      8.0      8.0
                        2011    283.0     17.0     14.0      7.0      4.0
                        2012    303.0     29.0      9.0      9.0      8.0
                        2013    408.0     34.0     18.0      4.0      1.0
                        2014    392.0     39.0     23.0      8.0      4.0
rape_declared_unfounded 2009      8.0      3.0      9.0      1.0      0.0
                        2010      8.0      5.0      2.0      2.0      1.0
                        2011     13.0      2.0      9.0      0.0      0.0
                        2012     12.0      4.0      2.0      0.0      0.0
                        2013      5.0      5.0      7.0      0.0      1.0
                        2014     13.0      4.0      2.0      0.0      0.0
rape_reported           2009    290.0     62.0     34.0      9.0      2.0
                        2010    272.0     75.0     22.0     10.0      9.0
                        2011    296.0     19.0     23.0      7.0      4.0
                        2012    315.0     33.0     11.0      9.0      8.0
                        2013    413.0     39.0     25.0      4.0      2.0
                        2014    405.0     43.0     25.0      8.0      4.0
unfounded_pct           2009      2.8      4.8     26.5     11.1      0.0
                        2010      2.9      6.7      9.1     20.0     11.1
                        2011      4.4     10.5     39.1      0.0      0.0
                        2012      3.8     12.1     18.2      0.0      0.0
                        2013      1.2     12.8     28.0      0.0     50.0
                        2014      3.2      9.3      8.0      0.0      0.0

## State Totals

In [11]:
state_annual_totals = agency_totals_by_year.groupby(agency_totals_by_year.index.str.slice(0, 2)).sum()[[
    "rape_declared_actual", "rape_declared_unfounded", "rape_reported"
]]
print("{0} states and territories".format(len(state_annual_totals)))
state_annual_totals.sort_values([("rape_declared_unfounded", 2014), ("rape_declared_actual", 2014)])

55 states and territories


rape_declared_actual                                           \
year                     2009    2010    2011    2012    2013     2014   
ori_code                                                                 
CZ                        0.0     0.0     0.0     0.0     0.0      0.0   
PR                        0.0     0.0     0.0     0.0     0.0      0.0   
GM                        0.0     0.0     0.0    61.0   106.0    117.0   
VT                      131.0   138.0   144.0   127.0   135.0    117.0   
RI                      296.0   298.0   319.0   292.0   449.0    366.0   
ND                      271.0   240.0   257.0   274.0   341.0    378.0   
DE                      394.0   326.0   307.0   249.0   399.0    390.0   
SD                      446.0   365.0   474.0   563.0   473.0    455.0   
WV                      427.0   336.0   350.0   390.0   596.0    506.0   
MT                      350.0   325.0   360.0   386.0   442.0    522.0   
NH                      362.0   382.0   403.0   421.0   628.0    553.0   
ID                      572.0   531.0   443.0   496.0   659.0    631.0   
IA                      876.0   857.0   842.0   868.0  1073.0   1116.0   
AR                     1315.0  1265.0  1215.0  1209.0  1392.0   1664.0   
SC                     1654.0  1535.0  1676.0  1709.0  2216.0   2168.0   
VA                     1571.0  1578.0  1589.0  1504.0  2349.0   2408.0   
TN                     2019.0  2173.0  2094.0  2042.0  2490.0   2583.0   
CO                     2249.0  2187.0  2257.0  2100.0  2902.0   3045.0   
IL                      121.0  2922.0  2933.0  2767.0  3908.0   4168.0   
MI                     4439.0  4631.0  4318.0  4545.0  6548.0   6248.0   
FL                     5494.0  5369.0  5273.0  5254.0  6737.0   7104.0   
VI                        0.0     0.0     0.0     0.0     0.0     54.0   
UT                      927.0   973.0   890.0   961.0  1464.0   1451.0   
DC                      150.0   187.0   173.0   236.0   395.0    472.0   
KY                     1448.0  1375.0  1477.0  1289.0  1628.0   1549.0   
HI                      385.0   377.0   353.0   279.0   235.0    288.0   
CT                      657.0   595.0   689.0   933.0   789.0    794.0   
WA                     2509.0  2569.0  2295.0  2212.0  2353.0   2529.0   
OR                     1193.0  1219.0  1230.0  1146.0  1156.0   1217.0   
WY                      171.0   161.0   144.0   150.0   140.0    165.0   
ME                      375.0   390.0   393.0   372.0   398.0    390.0   
NB                      590.0   647.0   673.0   681.0   663.0    692.0   
MA                     1722.0  1766.0  1619.0  1621.0  2139.0   2005.0   
MS                      828.0   761.0   666.0   648.0   539.0    528.0   
KS                     1133.0  1134.0  1109.0  1101.0  1186.0   1416.0   
OH                     3871.0  3500.0  3467.0  3625.0  3913.0   4638.0   
NJ                     1041.0   980.0  1005.0  1033.0   858.0    952.0   
NV                     1023.0   957.0   879.0   931.0  1089.0   1354.0   
AK                      509.0   528.0   434.0   579.0   919.0    764.0   
WI                     1114.0  1176.0  1176.0  1212.0  1516.0   1419.0   
NM                     1017.0   944.0   837.0   929.0  1052.0   1064.0   
LA                     1268.0  1190.0  1201.0  1102.0  1263.0   1339.0   
IN                     1508.0  1633.0  1601.0  1508.0  1756.0   1762.0   
OK                     1533.0  1456.0  1414.0  1617.0  1822.0   1860.0   
MN                     2240.0  2237.0  2111.0  2079.0  2002.0   1997.0   
MO                     1605.0  1435.0  1466.0  1522.0  2305.0   2398.0   
AL                     1466.0  1113.0  1326.0  1265.0  1960.0   1884.0   
AZ                     2225.0  2077.0  2385.0  2161.0  2297.0   3175.0   
MD                     1156.0  1228.0  1196.0  1236.0  1159.0   1141.0   
GA                     2238.0  2063.0  2026.0  2111.0  1974.0   2142.0   
NC                     2234.0  1890.0  1913.0  1937.0  1718.0   1659.0   
PA           

Because some states do not collect or publish information on unfounded rape cases, we ignored all states where the total number of "unfounded" rapes was 0 for any single year when calculating the national average.

In [12]:
states_reporting_zero_unfoundeds = state_annual_totals[
    (state_annual_totals["rape_declared_unfounded"].min(axis=1) == 0)
].index

In [13]:
print("{0} states and territories had at least one year with 0 rape cases declared 'unfounded' in the UCR report: {1}"\
    .format(len(states_reporting_zero_unfoundeds), " • ".join(states_reporting_zero_unfoundeds)))

22 states and territories had at least one year with 0 rape cases declared 'unfounded' in the UCR report: AR • CO • CZ • DE • FL • GM • IA • ID • IL • MI • MT • ND • NH • PR • RI • SC • SD • TN • VA • VI • VT • WV


## National Percentage Of "Unfounded" Cases By Year

*Note:* In determining the percentage of "unfounded" cases by year we excluded all states where there were 0 "unfounded" rape cases reported in any single year as discussed above.

In [14]:
national_annual_totals = state_annual_totals[
    ~state_annual_totals.index.isin(states_reporting_zero_unfoundeds)
].sum().unstack(level=0)
national_annual_totals["unfounded_pct"] = (100 * \
    national_annual_totals["rape_declared_unfounded"] / national_annual_totals["rape_reported"]).round(1)
national_annual_totals

,rape_declared_actual,rape_declared_unfounded,rape_reported,unfounded_pct
year,,,,
2009,61317.0,4869.0,66186.0,7.4
2010,58732.0,4358.0,63090.0,6.9
2011,57353.0,4117.0,61470.0,6.7
2012,58182.0,4325.0,62507.0,6.9
2013,62473.0,4604.0,67077.0,6.9
2014,73795.0,5324.0,79119.0,6.7


In [15]:
national_overall_unfounded_pct = (100 * \
    national_annual_totals["rape_declared_unfounded"].sum() / national_annual_totals["rape_reported"].sum()).round(1)

In [16]:
print("The national average 'unfounded' percentage during the entire time span was {0}%."\
          .format(national_overall_unfounded_pct))

The national average 'unfounded' percentage during the entire time span was 6.9%.


## Percentage of "Unfounded" Cases By Agency, Overall

In [17]:
agency_totals = pd.DataFrame({
    "rape_declared_actual": agency_totals_by_year["rape_declared_actual"].sum(axis=1),
    "rape_declared_unfounded": agency_totals_by_year["rape_declared_unfounded"].sum(axis=1),
    "rape_reported": agency_totals_by_year["rape_reported"].sum(axis=1),
})
agency_totals["unfounded_pct"] = (100 * \
    agency_totals["rape_declared_unfounded"] / agency_totals["rape_reported"]).round(1)
agency_totals.head()

,rape_declared_actual,rape_declared_unfounded,rape_reported,unfounded_pct
ori_code,,,,
AK00101,1932.0,59.0,1991.0,3.0
AK00102,248.0,23.0,271.0,8.5
AK00103,109.0,31.0,140.0,22.1
AK00104,44.0,3.0,47.0,6.4
AK00105,27.0,2.0,29.0,6.9


## A Look At The Baltimore City Police Department

The city of Baltimore had a high "unfounded" rate in 2009, but in recent years it has been below national average.

In [18]:
agencies.loc["MDBPD00"]

NAME             BALTIMORE POLICE DEPARTMENT
STATENAME                           MARYLAND
COUNTYNAME                  BALTIMORE (CITY)
LG_NAME                       BALTIMORE CITY
ADDRESS_NAME     BALTIMORE POLICE DEPARTMENT
ADDRESS_CITY                       BALTIMORE
ADDRESS_STATE                             MD
Name: MDBPD00, dtype: object

In [19]:
agency_totals_by_year.loc["MDBPD00"].unstack()

year,2009,2010,2011,2012,2013,2014
rape_declared_actual,158.0,265.0,341.0,317.0,298.0,245.0
rape_declared_unfounded,72.0,26.0,6.0,2.0,5.0,0.0
rape_reported,230.0,291.0,347.0,319.0,303.0,245.0
unfounded_pct,31.3,8.9,1.7,0.6,1.7,0.0


## A Look At The Baltimore County Police Department

Baltimore County's "unfounded" rate for 2009-14 is more than one-third. It has had a consistently high yearly rate between 2009 and 2014.

In [20]:
agencies.loc["MD00301"]

NAME             BALTIMORE COUNTY POLICE DEPARTMENT
STATENAME                                  MARYLAND
COUNTYNAME                                BALTIMORE
LG_NAME                            BALTIMORE COUNTY
ADDRESS_NAME     BALTIMORE COUNTY POLICE DEPARTMENT
ADDRESS_CITY                                 TOWSON
ADDRESS_STATE                                    MD
Name: MD00301, dtype: object

In [21]:
agency_totals_by_year.loc["MD00301"].unstack()

year,2009,2010,2011,2012,2013,2014
rape_declared_actual,152.0,126.0,142.0,145.0,116.0,93.0
rape_declared_unfounded,85.0,57.0,52.0,79.0,86.0,43.0
rape_reported,237.0,183.0,194.0,224.0,202.0,136.0
unfounded_pct,35.9,31.1,26.8,35.3,42.6,31.6


## Agencies With A High Percentage Of "Unfounded" Rape Reports

*Note:* We used a cutoff of a minimum of 250 cases and more than 20% of cases "unfounded" for this statistic in order to focus on larger agencies with a high percentage of "unfounded" rape reports.

In [22]:
depts_with_high_unfounded_rates = agency_totals[
    (agency_totals["rape_reported"] >= 250) &
    (agency_totals["unfounded_pct"] >= 20)
]

In [23]:
print("{0} agencies handled at least 250 rape reports between 2009 and 2014 and had more than 20 percent deemed unfounded."\
    .format(len(depts_with_high_unfounded_rates)))

15 agencies handled at least 250 rape reports between 2009 and 2014 and had more than 20 percent deemed unfounded.


Along with Baltimore County, other agencies with a high "unfounded" rate during this time period included the Dallas, Pittsburgh, and Oxnard, CA Police Departments. See below for all 15 agencies that met the 250 cases and at least 20% "unfounded" criteria.

In [24]:
depts_with_high_unfounded_rates\
    .join(agencies[["NAME"]])\
    .sort_values(["unfounded_pct"], ascending=False)

,rape_declared_actual,rape_declared_unfounded,rape_reported,unfounded_pct,NAME
CA05604,132.0,152.0,284.0,53.5,OXNARD POLICE DEPARTMENT
AZ00725,251.0,214.0,465.0,46.0,SCOTTSDALE POLICE DEPT
TX10115,383.0,226.0,609.0,37.1,PASADENA POLICE DEPARTMENT
TX05712,365.0,206.0,571.0,36.1,GRAND PRAIRIE POLICE DEPARTMENT
MD00301,774.0,402.0,1176.0,34.2,BALTIMORE COUNTY POLICE DEPARTMENT
PAPPD00,465.0,199.0,664.0,30.0,PITTSBURGH POLICE DEPARTMENT
TX01100,195.0,72.0,267.0,27.0,BASTROP COUNTY SHERIFF'S OFFICE
NC04102,396.0,144.0,540.0,26.7,GREENSBORO POLICE DEPARTMENT
AL04701,479.0,160.0,639.0,25.0,HUNTSVILLE POLICE DEPARTMENT
MD01604,710.0,223.0,933.0,23.9,MONTGOMERY COUNTY POLICE DEPARTMENT


The Baltimore County Police Department had the highest "unfounded" rate of any police department with at least 1,000 cases reported between 2009 and 2014.

In [25]:
large_depts_unfounded_rates = agency_totals[
    (agency_totals["rape_reported"] >= 1000)
]

In [26]:
print("""
{0} departments reported at least 1,000 total rape cases between 2009 and 2014 of those {1} reported no
'unfounded' rapes, while {2} reported at least one."\
""".strip().format(
        len(large_depts_unfounded_rates),
        len(large_depts_unfounded_rates[
                large_depts_unfounded_rates["rape_declared_unfounded"] == 0]),
        len(large_depts_unfounded_rates[
                large_depts_unfounded_rates["rape_declared_unfounded"] != 0])))

61 departments reported at least 1,000 total rape cases between 2009 and 2014 of those 19 reported no
'unfounded' rapes, while 42 reported at least one."


In [27]:
large_depts_unfounded_rates\
    .join(agencies[["NAME"]])\
    .sort_values(["unfounded_pct"], ascending=False)\
    .head(10)

,rape_declared_actual,rape_declared_unfounded,rape_reported,unfounded_pct,NAME
MD00301,774.0,402.0,1176.0,34.2,BALTIMORE COUNTY POLICE DEPARTMENT
TXDPD00,3228.0,943.0,4171.0,22.6,DALLAS POLICE DEPARTMENT
MD01721,980.0,248.0,1228.0,20.2,PRINCE GEORGE'S COUNTY POLICE DEPARTMENT
CA00109,1506.0,322.0,1828.0,17.6,OAKLAND POLICE DEPARTMENT
LANPD00,961.0,138.0,1099.0,12.6,NEW ORLEANS POLICE DEPARTMENT
NY03030,7424.0,937.0,8361.0,11.2,NEW YORK CITY POLICE DEPARTMENT
NB05501,911.0,110.0,1021.0,10.8,LINCOLN POLICE DEPT
TXSPD00,3876.0,437.0,4313.0,10.1,SAN ANTONIO POLICE DEPARTMENT
KY05680,1223.0,131.0,1354.0,9.7,LOUISVILLE METRO POLICE DEPARTMENT
AZ00717,1005.0,103.0,1108.0,9.3,MESA POLICE DEPARTMENT


----------

----------

----------